In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
 # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print('----------START GPU----------')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            print('----------END GPU----------')
    except RuntimeError as e:
 # Memory growth must be set before GPUs have been initialized
        print(e)

In [2]:
import skimage.io
import skimage.color
import skimage.transform
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import cv2
import pandas as pd
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.layers import Activation
from keras import backend as K
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D

In [ ]:
#Initialize a list of paths for images
imagepaths = []
import os
for dirname, _, filenames in os.walk('1125data'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        #print(os.path.join(dirname, filename))
        if path.endswith("png"):
            imagepaths.append(path)
            
print(len(imagepaths))

In [ ]:
data_dir = './1125data/'
classes = os.listdir(data_dir)
data = []
for cls in classes:
    files = os.listdir(data_dir+cls)
    for f in files:        
        img = skimage.io.imread(data_dir+cls+"/"+f)
        img =cv2.resize(img,(224, 224))
        data.append({
            'x':img,
            'y':cls
        })        
random.shuffle(data)

In [ ]:
X = [d['x'] for d in data]
y = [d['y'] for d in data]

y_dic = list(np.unique(y))
y = [y_dic.index(v) for v in y]
x_train = np.array(X[:int(len(X)*0.8)])
y_train = np.array(y[:int(len(X)*0.8)])

x_test = np.array(X[int(len(X)*0.8):])
y_test = np.array(y[int(len(X)*0.8):])

In [ ]:
batch_size = 32#batchsize：批大小。在深度学习中，一般采用SGD训练，即每次训练在训练集中取batchsize个样本训练；

num_classes = len(classes)
epochs =  10

## input image dimensions
img_rows, img_cols =224,224

num_classes

In [ ]:
#keras.utils.to_categorical()
def extend_channel(data):
    if K.image_data_format() == 'channels_first':
        data = data.reshape(data.shape[0], 1, img_rows, img_cols)
    else:
        data = data.reshape(data.shape[0], img_rows, img_cols, 1)
        
    return data

x_train = extend_channel(x_train)
x_test = extend_channel(x_test)

input_shape = x_train.shape[1:]


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train2 = keras.utils.to_categorical(y_train, num_classes)
y_test2 = keras.utils.to_categorical(y_test, num_classes)
input_shape

np.set_printoptions(threshold=np.inf)

In [ ]:
from keras.callbacks import ReduceLROnPlateau

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.0001)

In [ ]:
# from keras.applications import ResNet50

# height = 150 #dimensions of image
# width = 150
# channel = 3 #RGB

# # Create pre-trained ResNet50 without top layer
# model = ResNet50(include_top=False, weights="imagenet", input_shape=(height, width, channel))
# # Get the ResNet50 layers up to res5c_branch2c
# model = Model(input=model.input, output=model.get_layer('res5c_branch2c').output)

# model.trainable = False
# for layer in model.layers:
#     layer.trainable = False

# model = Sequential(model.layers)

In [3]:
# # model = ResNet50(input_shape = (150,150,1),weights=None,classes=10)
  
model = Sequential()
    
model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(224,224,3),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,2),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000,activation='softmax'))



In [4]:
# model = Sequential()
# model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu', input_shape = (224,224,1)))
# model.add(ZeroPadding2D((1, 1)))
# model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(ZeroPadding2D((1, 1)))
# model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
# model.add(ZeroPadding2D((1, 1)))
# model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same',activation ='relu'))
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 
# model.add(ZeroPadding2D((1, 1)))
# model.add(Conv2D(filters =256, kernel_size = (3,3),padding = 'Same',activation ='relu'))
# model.add(ZeroPadding2D((1, 1)))
# model.add(Conv2D(filters =256, kernel_size = (3,3),padding = 'Same',activation ='relu'))
# model.add(ZeroPadding2D((1, 1)))
# model.add(Conv2D(filters =256, kernel_size = (3,3),padding = 'Same',activation ='relu'))
# model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))


# model.add(Flatten())
# model.add(Dense(256))
# model.add(Activation('relu'))
# model.add(Dropout(0.1))
# model.add(Activation('relu'))
# model.add(Dense(512))
# model.add(Dropout(0.1))
# model.add(Dense(10, activation = "softmax"))

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])


In [ ]:
import tensorflow as tf
# 必须要下面这行代码
tf.compat.v1.disable_eager_execution()
print(tf.__version__)
 
# 我自己使用的函数
def get_flops_params():
    sess = tf.compat.v1.Session()
    graph = sess.graph
    flops = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
    params = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    print('FLOPs: {};    Trainable params: {}'.format(flops.total_float_ops, params.total_parameters))
 
 
# 网上推荐的
# sess = tf.compat.v1.Session()
# graph = sess.graph
# stats_graph(graph)
def stats_graph(graph):
    flops = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
    # print('FLOPs: {}'.format(flops.total_float_ops))
    params = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    # print('Trainable params: {}'.format(params.total_parameters))
    print('FLOPs: {};    Trainable params: {}'.format(flops.total_float_ops, params.total_parameters))
 
 
def get_flops(model):
    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    # We use the Keras session graph in the call to the profiler.
    flops = tf.compat.v1.profiler.profile(graph=tf.compat.v1.keras.backend.get_session().graph, run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops  # Prints the "flops" of the model.
 
# 必须使用tensorflow中的keras才能够获取到FLOPs, 模型中的各个函数都必须使用tensorflow.keras中的函数，和keras混用会报错

# 获取模型每一层的参数详情
model.summary()
# 获取模型浮点运算总次数和模型的总参数
get_flops_params()

In [7]:
import keras
def time_per_layer(model):
    new_model = model

    times = np.zeros((len(model.layers), 2))
    inp = np.ones((224, 224, 1))

    for i in range(1, len(model.layers)):
        new_model = keras.models.Model(inputs=[model.input], outputs=[model.layers[-i].output])

        # new_model.summary()
        new_model.predict(inp[None, :, :, :])

        t_s = time.time()
        new_model.predict(inp[None, :, :, :])
        t_e2 = time.time() - t_s

        times[i, 1] = t_e2
        del new_model

    for i in range(0, len(model.layers) - 1):
        times[i, 0] = abs(times[i + 1, 1] - times[i, 1])

    times[-1, 0] = times[-1, 1]


    return times


from net_flops import net_flops
net_flops(model,table=True)

               Layer Name |      Input Shape |     Output Shape |      Kernel Size |          Filters | Strides |  FLOPS
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
                   conv2d |    [224, 224, 3] |   [224, 224, 64] |           (3, 3) |               64 | (1, 1) | 173408256.0000
                 conv2d_1 |   [224, 224, 64] |   [224, 224, 64] |           (3, 3) |               64 | (1, 1) | 3699376128.0000
            max_pooling2d |   [224, 224, 64] |     ['', '', ''] |           (2, 2) |           [0, 0] | (2, 2) | 3211264.0000
                 conv2d_2 |   [112, 112, 64] |  [112, 112, 128] |           (3, 2) |              128 | (1, 1) | 1233125376.0000
                 conv2d_3 |  [112, 112, 128] |  [112, 112, 128] |           (3, 3) |              128 | (1, 1) | 3699376128.0000
          max_pooling2d_1 |  [112, 112, 128] |     ['', '', ''] |  

In [6]:

import time
import matplotlib.pyplot as plt
times = time_per_layer(model)

plt.style.use('ggplot')
x = [model.layers[-i].name for i in range(1,len(model.layers))]
#x = [i for i in range(1,len(model.layers))]
g = [times[i,0] for i in range(1,len(times))]
x_pos = np.arange(len(x))
plt.bar(x, g, color='#7ed6df')
plt.xlabel("Layers")
plt.ylabel("Processing Time")
plt.title("Processing Time of each Layer")
plt.xticks(x_pos, x,rotation=90)
print('Sum of all the layers time(s):')
print(sum(g))
plt.show()

ValueError: in user code:

    C:\Users\苏鸣\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    C:\Users\苏鸣\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\苏鸣\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\苏鸣\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\苏鸣\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1122 predict_step  **
        return self(x, training=False)
    C:\Users\苏鸣\AppData\Roaming\Python\Python37\site-pack

In [ ]:
history=model.fit(x_train, y_train2,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test2))

In [ ]:
model.save_weights('weights_train_model1129VGG16.h5')
model.save('train_model1129VGG16.h5')

In [ ]:
score = model.evaluate(x_test, y_test2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.savefig("1129VGG16.png")